In [79]:
import pandas as pd
from pathlib import Path
import numpy as np

In [10]:
spy_df = pd.read_csv(Path('Resources_Will/HistoricalPrices.csv'), index_col='Date', parse_dates=True, infer_datetime_format=True)

In [11]:
spy_df.columns

Index([' Open', ' High', ' Low', ' Close', ' Volume'], dtype='object')

In [12]:
spy_df = spy_df.drop(columns=[' Open',' High',' Low',' Volume'])
spy_df = spy_df.rename(columns = {' Close':'Close'})

In [13]:
spy_df.index

DatetimeIndex(['2023-06-28', '2023-06-27', '2023-06-26', '2023-06-23',
               '2023-06-22', '2023-06-21', '2023-06-20', '2023-06-16',
               '2023-06-15', '2023-06-14',
               ...
               '2000-04-13', '2000-04-12', '2000-04-11', '2000-04-10',
               '2000-04-07', '2000-04-06', '2000-04-05', '2000-04-04',
               '2000-04-03', '2000-03-31'],
              dtype='datetime64[ns]', name='Date', length=5847, freq=None)

In [14]:
spy_df = spy_df.groupby(by=[spy_df.index.year, spy_df.index.month]).mean('Close')

In [15]:
#list comprehension to reformat multi-index 
lst_spy = [(f'{k}/{j}') for j,k in spy_df.index ]
spy_df.index = lst_spy

spy_df.index = pd.to_datetime(spy_df.index)
spy_df = spy_df.reset_index()
spy_df['index'] = spy_df['index'].apply(lambda x: x.strftime('%Y-%m'))
spy_df.set_index('index', inplace=True)
spy_df


,Close
index,
2000-03,150.375000
2000-04,146.231911
2000-05,142.080250
2000-06,146.521318
2000-07,147.503905
...,...
2023-02,407.072632
2023-03,396.023913
2023-04,410.921053


https://www.macrotrends.net/2637/lumber-prices-historical-chart-data

In [16]:
lumber_df = pd.read_csv(Path('Resources_Will/lumber-prices-historical-chart-data.csv'), index_col='date', parse_dates=True, infer_datetime_format=True)

In [17]:
#calculate mean value for each month
lumber_df = lumber_df.groupby(by=[lumber_df.index.year, lumber_df.index.month]).mean()

In [18]:
#list comprehension to reformat multi-index 
lumber_lst = [(f'{k}/{j}') for j,k in lumber_df.index ]
lumber_df.index = lumber_lst

#make index datetime, reset the index so I can remove days, and put back into index
lumber_df.index = pd.to_datetime(lumber_df.index) 
lumber_df = lumber_df.reset_index()
lumber_df['index'] = lumber_df['index'].apply(lambda x: x.strftime('%Y-%m'))
lumber_df.set_index('index', inplace=True)
lumber_df.dropna(inplace=True)

In [20]:
lumber_df = lumber_df.rename(columns={'value':'lumber'})

In [145]:
lumber_df

,value
index,
1996-01,290.540909
1996-02,301.022000
1996-03,302.995238
1996-04,317.145714
1996-05,357.909091
...,...
2023-02,422.865000
2023-03,387.356522
2023-04,384.540000


In [141]:
vix_df = pd.read_csv(Path('Resources_Will/VIXCLS.csv'), index_col='DATE', parse_dates=True, infer_datetime_format=True, na_values='.')
vix_df = vix_df.ffill()

In [142]:
vix_df = vix_df.groupby(by=[vix_df.index.year, vix_df.index.month])['VIXCLS'].mean()

In [144]:
lst_vix = [(f'{k}/{j}') for j,k in vix_df.index ]
vix_df.index = lst_vix

vix_df.index = pd.to_datetime(vix_df.index)
vix_df = vix_df.reset_index()
vix_df['index'] = vix_df['index'].apply(lambda x: x.strftime('%Y-%m'))
vix_df.set_index('index', inplace=True)
vix_df

,VIXCLS
index,
2000-01,23.033333
2000-02,23.710000
2000-03,22.718261
2000-04,27.098500
2000-05,26.290435
...,...
2023-02,20.120500
2023-03,21.644348
2023-04,17.847000


In [146]:
master = pd.concat([vix_df, lumber_df, spy_df], axis=1)

In [149]:
master = master.dropna()

In [163]:
master = master.rename(columns={'VIXCLS':'VIX',' value':'Lumber','Close':'SPY'})

In [166]:
master = master.reset_index()

In [170]:
master['index'] = pd.to_datetime(master['index'])

In [171]:
master

,index,VIX,Lumber,SPY
0,2000-03-01,22.718261,333.900000,150.375000
1,2000-04-01,27.098500,311.249474,146.231911
2,2000-05-01,26.290435,291.777273,142.080250
3,2000-06-01,21.540000,288.352727,146.521318
4,2000-07-01,19.890000,258.655000,147.503905
...,...,...,...,...
275,2023-02-01,20.120500,422.865000,407.072632
276,2023-03-01,21.644348,387.356522,396.023913
277,2023-04-01,17.847000,384.540000,410.921053
278,2023-05-01,17.636087,345.117391,413.880909
